In [1]:
import psycopg2
from sqlalchemy import create_engine

import os
import pandas as pd

def pgsql_conn(engine):
    if engine == 'psycopg2':
        conn = psycopg2.connect(
            database = os.environ['POSTGRES_DB'], 
            user = os.environ['POSTGRES_USER'],
            host = os.environ['POSTGRES_HOST'],
            password = os.environ['POSTGRES_PASSWORD'],
            port = os.environ['POSTGRES_PORT']
        )
    elif engine == 'sqlalchemy' :
        conn_string = 'postgresql://{}:{}@{}:{}/{}'.format(
            os.environ['POSTGRES_USER']
            , os.environ['POSTGRES_PASSWORD']
            , os.environ['POSTGRES_HOST']
            , os.environ['POSTGRES_PORT']
            , os.environ['POSTGRES_DB']
        )
        db = create_engine(conn_string) 
        conn = db.connect()
    else:
        raise ValueError('engine not contain in (psycopg2,sqlalchemy)')
    return conn

def executeScriptsFromFile(filename):
    conn = pgsql_conn('psycopg2')
    conn.autocommit = True
    cursor = conn.cursor() 
    fd = open(filename, 'r')
    sqlFile = fd.read()
    fd.close()
    sqlCommands = sqlFile.split(';')[:-1]
    for command in sqlCommands:
        cursor.execute(command)   
    conn.close()
    return sqlCommands

def pgsql_pd_insert(df, table_name, if_exists='append', index=False):
    conn = pgsql_conn('sqlalchemy')
    df.to_sql(table_name, con=conn, if_exists=if_exists, index=index)
    conn.close()
    return
    
def pgsql_pd_read(sql):
    conn = pgsql_conn('sqlalchemy')
    df = pd.read_sql_query(sql, con = conn)
    conn.close()
    return df

# Init postgres DB

In [2]:
log_init = executeScriptsFromFile('soruceCode.sql')

# Part A : EDA Process

In [3]:
df = pd.read_csv('bestsellers with categories.csv')

## A1 : Count row

In [4]:
df.tail() # n = 550

,Name,Author,User Rating,Reviews,Price,Year,Genre
545,Wrecking Ball (Diary of a Wimpy Kid Book 14),Jeff Kinney,4.9,9413,8,2019,Fiction
546,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2016,Non Fiction
547,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2017,Non Fiction
548,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2018,Non Fiction
549,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2019,Non Fiction


## A2 : Check null

In [5]:
df[df.isna().any(axis=1)]

,Name,Author,User Rating,Reviews,Price,Year,Genre
518,"Unbroken: A World War II Story of Survival, Re...",Laura Hillenbrand,NaN,29673,13,2014,Non Fiction
521,Unfreedom of the Press,Mark R. Levin,NaN,5956,11,2019,Non Fiction
542,Wonder,R. J. Palacio,NaN,21625,9,2015,Fiction


## A3 : Check Zero

In [6]:
print("User Rating :",len(df[df['User Rating']==0]))
print("Reviews :",len(df[df['Reviews']==0]))
print("Price :",len(df[df['Price']==0]))
print("Year :",len(df[df['Year']==0]))

User Rating : 0
Reviews : 0
Price : 12
Year : 1


# Part B : ETL process

## B1 : Extract by specific input

In [7]:
expect_input = {
   'Name': 'str'
   , 'Author': 'str'
   , 'User Rating': 'float'
   , 'Reviews': 'int'
   , 'Price': 'int'
   , 'Year': 'int'
   , 'Genre': 'str'
}
raw_data = pd.read_csv('bestsellers with categories.csv'
                       , dtype = expect_input
                       , usecols = list(expect_input.keys())
                       , header = 0
                      )

## B2 : Transform & Load

### B2.1 : Change the column names to be clear & lower case

In [8]:
book_reviews = raw_data.rename(columns={
    'Name':'book_name'
    , 'Author':'writer_name'
    , 'User Rating':'user_rating'
    , 'Reviews':'reviews'
    , 'Price':'price'
    , 'Year':'year'
    , 'Genre':'category_name'
})

In [9]:
book_reviews.tail()

,book_name,writer_name,user_rating,reviews,price,year,category_name
545,Wrecking Ball (Diary of a Wimpy Kid Book 14),Jeff Kinney,4.9,9413,8,2019,Fiction
546,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2016,Non Fiction
547,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2017,Non Fiction
548,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2018,Non Fiction
549,You Are a Badass: How to Stop Doubting Your Gr...,Jen Sincero,4.7,14331,8,2019,Non Fiction


### B2.2 Update master table of category (category_list) on Postgres and Give category_id to book_reviews

In [10]:
category_list = book_reviews[['category_name']].drop_duplicates().reset_index(drop=True)
pgsql_pd_insert(category_list, "category_list")

category_list = pgsql_pd_read('SELECT "id" as category_id, "category_name" FROM pgdb.public."category_list"')
book_reviews = pd.merge(book_reviews, category_list, how='left',on=['category_name']).drop('category_name',axis=1)

In [11]:
category_list.head()

,category_id,category_name
0,1,Non Fiction
1,2,Fiction


### B2.3 Update master table of writer (writer_list) on Postgres and Give writer_id to book_reviews

In [12]:
writer_list = book_reviews[['writer_name']].drop_duplicates().reset_index(drop=True)
pgsql_pd_insert(writer_list, "writer_list")

writer_list = pgsql_pd_read('SELECT "id" as writer_id, "writer_name" FROM pgdb.public."writer_list"')
book_reviews = pd.merge(book_reviews, writer_list, how='left',on=['writer_name']).drop('writer_name',axis=1)

In [13]:
writer_list.tail()

,writer_id,writer_name
243,244,Cheryl Strayed
244,245,Ken Follett
245,246,Geneen Roth
246,247,R. J. Palacio
247,248,Jen Sincero


### B2.4 Update master table of book (book_list) on Postgres and Give book_id to book_reviews

In [14]:
book_list = book_reviews[['book_name','category_id','writer_id']].drop_duplicates().reset_index(drop=True)
pgsql_pd_insert(book_list, "book_list")

book_list = pgsql_pd_read('SELECT "id" as book_id, "book_name" FROM pgdb.public."book_list"')
book_reviews = pd.merge(book_reviews, book_list, how='left',on=['book_name']).drop('book_name',axis=1)
book_reviews = book_reviews.drop(['category_id','writer_id'],axis=1)

In [15]:
book_list.tail()

,book_id,book_name
346,347,Winter of the World: Book Two of the Century T...
347,348,Women Food and God: An Unexpected Path to Almo...
348,349,Wonder
349,350,Wrecking Ball (Diary of a Wimpy Kid Book 14)
350,351,You Are a Badass: How to Stop Doubting Your Gr...


### B2.5 Insert data to book_reviews table on Postgres

In [16]:
book_reviews.loc[book_reviews['year'] == 0,'year'] = None ## clean year = 0

In [17]:
pgsql_pd_insert(book_reviews, "book_reviews")

In [18]:
book_reviews.tail()

,user_rating,reviews,price,year,book_id
545,4.9,9413,8,2019.0,350
546,4.7,14331,8,2016.0,351
547,4.7,14331,8,2017.0,351
548,4.7,14331,8,2018.0,351
549,4.7,14331,8,2019.0,351


# PART C : Simple Analytic

In [19]:
ana_df = pgsql_pd_read('SELECT count(*) as Total_writer FROM pgdb.public."writer_list"')

In [20]:
ana_df

,total_writer
0,248


In [21]:
ana_df = pgsql_pd_read('SELECT year, count(*) as Total_review FROM pgdb.public."book_reviews" WHERE year is not null GROUP BY year')

In [22]:
ana_df

,year,total_review
0,2017,49
1,2009,50
2,2013,50
3,2016,50
4,2018,50
5,2014,50
6,2012,50
7,2015,50
8,2010,50
9,2019,50
